In [8]:
import pandas as pd
import cobra
import logging

In [9]:
def change_medium(model, sub_type, sub):
    '''
        {'r_1714'}; % D-glucose exchange
        {'r_1654'}; % ammonium exchange
        {'r_2005'}; % phosphate exchange
        {'r_2060'}; % phosphate exchange
    '''
    model_test = model.copy()
    medium = model_test.medium
    medium[sub.id] = 10.0
    if sub_type == 'C':
        medium['r_1714'] = 0.0
    elif sub_type == 'N':
        medium['r_1654'] = 0.0
    elif sub_type == 'P':
        medium['r_2005'] = 0.0
    elif sub_type == 'S':
        medium['r_2060'] = 0.0
    model_test.medium = medium
    return model_test

In [10]:
def add_exchange(model, metid):
    model_test = model.copy()
    try:
        rxn = model_test.add_boundary(model_test.metabolites.get_by_id(metid))
        logging.info('add {}'.format(rxn.reaction))
    except ValueError:
        met = model_test.metabolites.get_by_id(metid)
        print(metid)
        met.compartment = 'e'
        model_test.add_metabolites(met)
        rxn = model_test.add_boundary(met, type="exchange")
        logging.info('add {}'.format(rxn.reaction))
    return model_test, rxn

In [11]:
def simu(model):
    sol = model.optimize()
    if sol.objective_value > 1e-6:
        gng = 'G'
    else:
        gng = 'NG'
    return gng


logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s')
biolog = pd.read_table('./Biolog_Substrate.tsv')

In [12]:
model = cobra.io.read_sbml_model('./Yeast9.xml')
exp_simu = pd.DataFrame(columns=['Name_in_Model', 'id', 'Substrate_type', 'Growth_Biolog', 'Growth_Model'])
for m in model.metabolites:
    for i in range(len(biolog.index)):
        if m.name == biolog.loc[i, 'Name_in_Model']:
        #if m.name.split(' ')[0] == biolog.loc[i, 'Name_in_Model']:
            exp_simu.loc[i, 'id'] = m.id
            exp_simu.loc[i, 'Substrate_type'] = biolog.loc[i, 'Substrate_type']
            exp_simu.loc[i, 'Growth_Biolog'] = biolog.loc[i, 'Growth_Biolog']
            exp_simu.loc[i, 'Name_in_Model'] = biolog.loc[i, 'Name_in_Model']

exp_simu.index = range(len(exp_simu))


if __name__ == '__main__':
    for i in range(len(exp_simu.index)):
        model_test, rxn = add_exchange(model, exp_simu.loc[i, 'id'])
        model_final = change_medium(model_test, exp_simu.loc[i, 'Substrate_type'], rxn)
        gng = simu(model_final)
        logging.info('{}: {}'.format(str(exp_simu.loc[i, 'Name_in_Model']), gng))
        exp_simu.loc[i, 'Growth_Model'] = gng

    c = 0
    for i in range(len(exp_simu.index)):
        if exp_simu.loc[i, 'Growth_Biolog'] == exp_simu.loc[i, 'Growth_Model']:
            c += 1

    exp_simu.to_excel('./yeast9_sub_use.xlsx'.format(model.id))

2024-05-24 15:09:10,074 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:09:10,093 - sbml.py[line:1672] - DEBUG: Unexpected content format 'Saccharomyces cerevisiae - strain S288C'.
2024-05-24 15:09:17,990 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:09:18,522 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:18,522 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:18,522 - 3936220034.py[line:12] - INFO: add s_0065 <=> 
2024-05-24 15:09:18,537 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


s_0065


2024-05-24 15:09:19,056 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:19,071 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:19,703 - 3888050103.py[line:20] - INFO: (S)-lactate: G
2024-05-24 15:09:19,703 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:09:20,228 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:20,234 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:20,236 - 3936220034.py[line:12] - INFO: add s_0069 <=> 
2024-05-24 15:09:20,237 - model.py[line:209] - INFO: T

s_0069


2024-05-24 15:09:20,790 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:20,810 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:21,391 - 3888050103.py[line:20] - INFO: (S)-malate: G
2024-05-24 15:09:21,391 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:09:21,910 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:09:21,910 - 3936220034.py[line:5] - INFO: add s_4161 <=> 
2024-05-24 15:09:21,910 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:09:22,391 - boundary_types.py[line:68] - INFO: Compartment `e` sounds l

...

2024-05-24 15:15:53,924 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:15:53,940 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:15:54,509 - 3888050103.py[line:20] - INFO: Gly-Met: G
2024-05-24 15:15:54,523 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:15:55,057 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:15:55,057 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:15:55,072 - 3936220034.py[line:12] - INFO: add s_4087 <=> 
2024-05-24 15:15:55,074 - model.py[line:209] - INFO: The c

s_4087


2024-05-24 15:15:55,607 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:15:55,624 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:15:56,193 - 3888050103.py[line:20] - INFO: Gly-Met: G
2024-05-24 15:15:56,193 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:15:56,757 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:15:56,757 - 3936220034.py[line:5] - INFO: add s_4090 <=> 
2024-05-24 15:15:56,757 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:15:57,274 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like

s_4121


2024-05-24 15:16:23,472 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:16:23,489 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:16:24,006 - 3888050103.py[line:20] - INFO: Ala-Asp: G
2024-05-24 15:16:24,006 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:16:24,573 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-05-24 15:16:24,573 - 3936220034.py[line:5] - INFO: add s_4125 <=> 
2024-05-24 15:16:24,573 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-05-24 15:16:25,124 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like